Reflector Area Processing - Further Analysis
============================================



The analysis in "Reflector Area Processing - Testing" revealed that there are significant differences in all metrics except for area distribution shapes when comparing a part of a raw section versus the same part of section that had fractured grains (both actually-fractured as well as split up due to the thresholding method) manually reconstructed. These differences also signify that the raw data is not the most representative of actual reflector distributions, and hence some sort of gap-filling (whether manual or automatic) is required.

Due to the excessive time consumption of this manual gap filling, automatic gap filling is preferred for the full analysis. However the full analysis in `../section-scans-full/working.org` showed that there's the degree of automatic gap filling (via dilation-erosion) affects the outcome as well. In order to evaluate the different degrees of automatic gap filling, the following analysis is devised:

1.  Display the reflector patches without gap filling, with reflector patches of interest (i.e. with area greater than or equal to 5 px and lesser than or equal to 0.05 mm<sup>2</sup>) in one colour and the others in another colour.
2.  Run the automatic gap filling.
3.  Filter out the reflectors of interest after gap filling.
4.  Display the reflector patches of interest after automatic gap filling in one colour, and the other reflector patches after automatic gap filling in another colour.
5.  Visually inspect the difference in reflector patches before and after to see whether the automatic gap filling produces reasonable results (reconstructs a large number of grains that were clearly whole originally without artificially creating too many nonsensical large grains from grains that are clearly separate).



## Contour Area Slicing



This involves converting the contour area slicing method to display just contours with areas of interest described in `../section-scans-example/working.org` into a reusable method. This notebook should be run with dilate<sub>erode</sub> = each of {0,10,20} (after `../section-scans-full/working.org` is also run with each of those kernel sizes).



In [1]:
# Option to change.
dilate_erode = 0

from area_slicer import *
import os

import sys
sys.path.insert(0,os.path.join("..","section-scans-full"))
from util_funcs import *

sample = "M01"
# Declare conversion factor between pixels and millimeters.
pix2mm = 0.001
# Initialize grain filter by area (where the grain detections are specific to a sample *and* dilation erosion kernel size). The contour definitions are created after a dilation-erosion without removal of any grains (i.e. the previous method).
AS = AreaSlicer(sample,dilate_erode)
# Plot/highlight post-dilation-erosion grains of size between 5 microns^2 and 0.05 mm^2 onto the EDS map.
img = AS.visualise_patches([5*pix2mm**2,0.05],(0,255,0))
# Construct the path to save the image with highlighted grains of the size range using the previous dilate-erode method.
filtered_grains_img_file = os.path.join("imgs",f"{sample}-{str(dilate_erode)}-sliced.jpg")
# Save the image to disk if it doesn't already exist.
if not os.path.exists(filtered_grains_img_file):
    cv2.imwrite(filtered_grains_img_file,img)

Since area distributions are of the greatest focus, a section whose area distribution changed significantly with increasing (automatic) dilation-erosion (see `../section-scans-full/working.org`) was selected for an initial run of this analysis: M01.

Features of interest from the results are shown in figure [9](#org1c03b8a).

-   Well joined region:
    -   Where there's no dilation-erosion: reflector patches that are interpreted as separate and appear separate.
    -   Where there's some dilation-erosion: reflector patches that are interpreted as being one patch and appear as such. Embayed regions may or may not be filled in.
-   Badly joined region:
    -   Where there's no dilation-erosion: reflector patches that are interpreted as being one patch but appear separate.
    -   Where there's some dilation-erosion: reflector patches that are joined without being interpreted as being one patch, or are "well-joined" patches that have grown beyond the borders of the interpreted patch.

![img](./imgs/comparisons.png "Comparing the outputs of different degrees of automatic gap filling for parts of sample M01, showing a region that's been gap-filled well, and a region that's been very poorly gap-filled.")

Using a 10x10 px kernel for dilation-erosion appears to produce the best results, with individual patches being joined up without overjoining. However, there are still cases of overjoining, often with narrow bridges between grains that should be separate.

One major issue contributing to this is the joining of grains that have been defined as "noise" on the basis of their area covering less than 5 px (irrespective of whether they actually are or not). The result of this is the appearance of false grains - grains that enter the realm of signal (on the basis of area growing greater than 5 px) due to size growth related to the dilation-erosion process.

Therefore, it's proposed that grains interpreted as noise on the basis of their size are discarded **before** the dilation-erosion process takes place. This order is more sensible than previous since anything interpreted as "noise" shouldn't be processed as part of the actual signal. Furthermore, the dilation-erosion that should produce the best results (out of the ones tested) uses a 10x10 px kernel.

This requires a relatively major change to the processing's command flow, as captured by the following code:



In [1]:
import os

def load_unmerged_contours(sample):
    ''' Load unmodified grain contour definitions (both normal size and 2x upscaled). Note: grain contours are not the same as grain boundaries - grain contours are shrunk from the pixel boundaries by half a pixel.

    sample | :str: | Sample name to load the boundary definitions for.

    Returns: (:np.array:,:np.array:)
    '''
    base_data_file = os.path.join("..","..","DATASETS","RL_scans",f"contours-unmodified/{sample}.png")
    contours = np.load(base_data_file + ".npy",allow_pickle=True)
    larger_contours = np.load(base_data_file + "-larger.npy",allow_pickle=True)
    return contours,larger_contours

def find_areas(contours,larger_contours,pix2mm=None):
    ''' Compute the area of grain detections in pixels covered (not the area of the real-size boundary specification, which can include zero-width parts that are one-pixel wide).

    contours | :np.array: [<grain contour definitions>] | Array of contours that define grain boundaries.
    larger_contours | :np.array: [<grain contour definitions>] | Array of 2x upscaled contours that define grain boundaries.
    pix2mm | Numerical | Conversion factor of pixels to mm.

    Returns: :np.array:,:str:
    '''
    # Find area within contour for "real"-size grain detection contours.
    contour_areas = np.array(list(map(cv2.contourArea,contours)))
    # Find area within contour for 2x upscaled grain detection contours.
    larger_contour_areas = np.array(list(map(cv2.contourArea,larger_contours)))
    # Compute patch areas in pixels from these two (parallel) area arrays.
    patch_areas = (larger_contour_areas + 1 - 2 * contour_areas)/2
    # Check whether area unit conversion is desired.
    if pix2mm:
        # If so, convert to millimeters using the provided conversion factor.
        patch_areas = np.array(patch_areas) * (pix2mm**2)
        # Declare that areas are returned in the units of mm.
        units = "mm"
    else:
        # Otherwise declare the areas are returned in the units of px.
        units = "px"
    return patch_areas,units

def draw_contours(contours,img):
    ''' Draw contour definitions as white patches atop an image. The image should have the same dimensions as the one from which the contour definitions were extracted.

    contours | :np.array: | Array of contour definitions to draw over the image.
    img | :np.array: | Array of [R,G,B,A] "pixels" that have the same dimensions as the image from which the contours were extracted.

    Returns: :np.array:
    '''
    [cv2.fillPoly(img,[np.reshape(c,(c.shape[0],2))],color=(255,255,255,255)) for c in contours]
    return img
# Construct path to the JPG-ified full EDS map.
base_image = os.path.join("..","..","DATASETS","RL_scans",f"{sample}.jpg")
# Construct path to the image containing only white patches representing dilate-erode denoised grain detections, using the new merging method.
refined_grain_denoised_img = os.path.join("imgs",f"{sample}.png")
# Check whether the merged grains image exists (i.e. whether the new method needs to be run).
if not os.path.exists(refined_grain_denoised_img):
    # Load grain contours (both "real"- and 2x upscale sized).
    contours,larger_contours = load_unmerged_contours(sample)
    # Find grain areas (in mm^2).
    areas,units = find_areas(contours,larger_contours,pix2mm)
    # Filter the grain contours to remove grains smaller than 5 pixels (essentially the unconfident detections) - i.e. denoise the grain detections.
    size_filter = construct_minmax_filter(areas,5*pix2mm**2)
    contours = list_of_list_filter(contours,size_filter)
    # Load the EDS map.
    img = cv2.imread(base_image)
    # Create a blank image.
    blank_img = np.zeros([*img.shape[:2],4])
    # Draw the unmerged (no dilation-erosion processing) contours onto the blank image.
    contour_img = draw_contours(contours,blank_img)
    # Save this image.
    cv2.imwrite(refined_grain_denoised_img,contour_img)
else:
    # Otherwise just load the EDS map.
    img = cv2.imread(base_image)
# Initialize area processer that contains the denoised contour collection, and will be applying a dilate-erode processing using the previously specified kernel size.
AP = AreaProcessor(refined_grain_denoised_img,pix2mm,dilate_erode,load_dir=os.path.join("..","section-scans-full","filtered_data"))
# Load the grain contours after dilate-erode processing (both "real"-size and 2x upscaled).
contours,larger_contours = AP.load_contours()
# Initialize grain filter by area (where the grain detections are specific to a sample *and* dilation erosion kernel size).
AS = AreaSlicer(sample,dilate_erode,base_folder="imgs",AP=AP)
# Plot/highlight post-dilation-erosion grains of size between 5 microns^2 and 0.05 mm^2 onto the EDS map.
img = AS.visualise_patches([5*pix2mm**2,0.05],(0,255,0),img)
# Construct the path to save the image with highlighted grains of the size range after dilate-erode using the new method.
overlain_img = os.path.join("imgs",f"refined-{sample}-{str(dilate_erode)}-sliced.jpg")
# Save this image if it doesn't already exist.
if not os.path.exists(overlain_img):
    cv2.imwrite(overlain_img,img)

Features of interest from the refined results are shown in figure [2](#org28ec3e0).

![img](./imgs/refined-comparisons.png "Comparing the outputs of different degrees of automatic gap filling for parts of sample M01 when using the refined processing pathway, showing the same regions as in the previous figure.")

This new method not only reduces overjoining and development of badly-joined regions (in both 10x10 px and 20x20 px kernel dilation-erosion cases), but also avoids joining "noise" and creating false reflectors. In the top row, 10x10 px kernel dilation-erosion doesn't fully reconstruct the coarsest grain like 20x20 px, but that grain is very heavily embayed, and 10x10 px has the most well joined patches in the bottom row. Despite the issues with a 20x20 px kernel, visually comparing the bottom-right image (new method) in this image to the previous (old method) also reveals a significant improvement.

This refined method and a 10x10 px kernel will be used in `../section-scans-refined/working.org` to produce confidently-interpretable results.

